# Lab formativo 2

### En este laboratorio vamos entrenar un modelo para predicir la temperatura media dado un año, mes y ciudad

#### Dado las restricciones de poder de computo de Coursera, les recomendo descargar el notebook y ejecutar en vuestras computadoras o en google colab.

In [ ]:
# Descomentar al ejecutar en vuestros computadoras
# try:
#     import pandas as pd
#     import numpy as np
#     import torch
#     import torch.nn as nn
#     import torch.optim as optim
#     from sklearn.model_selection import train_test_split
#     from sklearn.preprocessing import StandardScaler, LabelEncoder
# except ModuloNotFoundError as err:
#     !pip install pandas numpy torch scikit-learn
#     import pandas as pd
#     import numpy as np
#     import torch
#     import torch.nn as nn
#     import torch.optim as optim
#     from sklearn.model_selection import train_test_split
#     from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder




In [ ]:
file_path = 'GlobalLandTemperaturesByCity.csv' # Está disponible en el Coursera, para que puedan trabajar desde sus computadoras.
# Load the dataset
df = pd.read_csv(file_path)
# Handle missing values
df = df.dropna()

country_count = df['Country'].value_counts()
country_count



### Para que sea más plausible entrenar nuestro modelo, elegiremos un país del conjunto de datos. Si observamos cómo están distribuidos los datos, podemos notar que están desbalanceados, lo que requeriría mucho más trabajo de manejo de datos.

En casos como este, generalmente tendríamos que aplicar técnicas de aumento de datos, como hicimos con el conjunto de datos Iris Flower, o tal vez hacer combinaciones con otros conjuntos de datos eliminando duplicados y obteniendo más ejemplos. También es posible reducir la cantidad de ejemplos en el conjunto de datos, pero no es suficiente simplemente mantener la misma cantidad de datos de manera arbitraria; debemos verificar si son de fechas similares, etc.

Para evitar todo ese trabajo, trabajaremos solo con el país que tiene más ejemplos, en este caso, India.

In [ ]:
df = df[df['Country'].isin(['India'])]
df

### Para que nuestro modelo pueda entender mejor los datos, normalizaremos la temperatura. Esto nos ayudará a obtener mejores resultados y a que el algoritmo calcule más rápido. También cambiaremos el formato de la fecha al formato mes año como columnas a parte.

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

scaler = MinMaxScaler()
df['AverageTemperature'] = scaler.fit_transform(df[['AverageTemperature']])

labelencoder = LabelEncoder()
df['City'] = labelencoder.fit_transform(df['City']) # Acá vamos crear mapeos para las ciudades, ejemplo ficticio Abohar = 0 ... Yelahanka = 35


df['Year'] = pd.DatetimeIndex(df['dt']).year

df = df[df['Year'] >= 1980] # Usaremos solo los datos posteriores al año 1980, esto por limitación de Ram de Coursera y para que el modelo termine de entrenar.

df['Month'] = pd.DatetimeIndex(df['dt']).month

print("\nTransformed DataFrame:")
df




## Aquí proporciono una versión condensada del conjunto de datos para agilizar la inferencia y el entrenamiento. Utilice exclusivamente esta versión para determinar qué hiperparámetros y configuraciones emplear.

In [ ]:

print("\nTransformed DataFrame:")
df_small = df.sample( df.size //100, replace=True)
df_small

Aquí definimos nuestro modelo. Cabe destacar que la configuración exacta de capas y neuronas no es obligatoria; siéntanse libres de modificar estos ajustes si creen que se pueden hacer mejoras.

Verán que estamos aplicando los conceptos aprendidos esta semana, incluyendo la normalización y el dropout.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TemperaturePredictor(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(2, 64)
        self.bn1 = nn.BatchNorm1d(64)
        
        self.fc2 = nn.Linear(64, 128)
        self.bn2 = nn.BatchNorm1d(128)
        
        self.fc3 = nn.Linear(128, 256)
        self.bn3 = nn.BatchNorm1d(256)
        
        self.fc4 = nn.Linear(256, 256)
        self.bn4 = nn.BatchNorm1d(256)

        self.fc5 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout(x)
        
        x = F.tanh(self.fc2(x))
        x = self.bn2(x)
        x = self.dropout(x)
        
        x = F.tanh(self.fc3(x))
        x = self.bn3(x)
        x = self.dropout(x)
        
        x = F.tanh(self.fc4(x))
        x = self.bn4(x)
        x = self.dropout(x)
        
        x = self.fc5(x)
        return x

Aquí definimos cuáles serán los datos de entrada y qué es lo que vamos a predecir. Además, para un mejor manejo de los datos, utilizaremos un DataLoader. Esta función nos permite entrenar en pequeños lotes, lo que facilita una convergencia del modelo más rápida.

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch

df_train = df[["Year", "Month"]]
df_target = df["AverageTemperature"]

X = torch.tensor(df_train.values, dtype=torch.float32)
y = torch.tensor(df_target.values, dtype=torch.float32).view(-1,1)

indices = torch.randperm(len(X))

X = X[indices]
y = y[indices]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.02, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



Tenemos 1 DataLoader para el entrenamiento y un para la validación.

Abajo procedemos a entrenar el modelo



In [ ]:
import torch
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR

criterion = nn.MSELoss()

model = TemperaturePredictor().to(device)

optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.00001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.7)
loss_values = []
accumulation_steps = 4
for epoch in range(2):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        # Move data to GPU
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss = loss / accumulation_steps  # Normalize loss
        loss.backward()
        loss_values.append(loss.item())
        # Update weights
        if (i+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        if (i+1) % 1000 == 0:
            print(loss.item())

    torch.cuda.empty_cache()
    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}, Loss activacion: {loss.item()} ")
    scheduler.step()


Por fim procedemos a validar el modelo

In [ ]:
model.eval()  # Set the model to evaluation mode
total_val_loss = 0

with torch.no_grad():
    for batch_X, batch_y in val_loader:
        # Forward pass
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)

        # Compute loss
        loss = criterion(outputs, batch_y)

        # Accumulate loss
        total_val_loss += loss.item()

average_val_loss = total_val_loss / len(val_loader)
print(f"Validation Loss: {average_val_loss}")

In [ ]:
def validate_regression_model_with_loader(model, val_loader, criterion, device):
    model.eval()
    total_val_loss = 0
    predictions = []
    true_labels = []  # To store true labels
    right = 0

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_val_loss += loss.item() * len(batch_y)
            
            # Store predictions and true labels
            predictions.extend(outputs.cpu().numpy())
            true_labels.extend(batch_y.cpu().numpy())

    average_val_loss = total_val_loss / len(val_loader.dataset)
    
    # Check the number of predictions within a certain tolerance level
    for i in range(len(predictions)):
        if true_labels[i] - 0.12 <= predictions[i] <= true_labels[i] + 0.12:
            right += 1

    accuracy_within_tolerance = right / len(predictions)
    #print(list(zip(predictions, true_labels)))
    print(list(zip(predictions[0], true_labels[0])))
    return average_val_loss, accuracy_within_tolerance

# Dummy criterion and device for demonstration
criterion = nn.MSELoss()

device='cpu'
# Run the validation loop
average_val_loss, accuracy_within_tolerance = validate_regression_model_with_loader(model, val_loader, criterion, device)
print(f"Average Validation Loss: {average_val_loss}")
print(f"Accuracy within Tolerance: {accuracy_within_tolerance * 100}%")



## ¿Cambia mi resultado si cambio el tamaño del lote (batch size)?
## ¿Y la tasa de aprendizaje (Learning Rate)?
## ¿Qué sucede si elimino las capas de normalización por lotes (Batch Normalization), reduzco el tamaño del lote y aumento la cantidad de épocas?